## <font color='darkblue'>Preface</font>
([course link](https://towardsdatascience.com/grid-search-vs-random-search-vs-bayesian-optimization-2e68f57c3c46)) <b><font size='3ptx'>Which hyperparameter tuning method is best?</font></b> <b>Hyperparameter tuning is an essential step in developing robust predictive models. After all, sticking with default parameters prevents models from achieving peak performance.</b>

This begs the question: what method is most fitting for finding the optimal hyperparameters for a given model?

Here, we delve into 3 popular approaches for hyperparameter tuning and determine which one is superior.

<a id='sect0'></a>
### <font color='darkgreen'>Agenda</font>
* <b><font size='3ptx'><a href='#sect1'>Introduction of 3 popular approaches for hyperparameter tuning</a></font></b>
* <b><font size='3ptx'><a href='#sect2'>Case Study</a></font></b>
* <b><font size='3ptx'><a href='#sect3'>Which method is the best?</a></font></b>

<a id='sect1'></a>
## <font color='darkblue'>Introduction of 3 popular approaches for hyperparameter tuning</font>
* <b><a href='#sect1_1'>Grid search</a></b>
* <b><a href='#sect1_2'>Random search</a></b>
* <b><a href='#sect1_3'>Bayesian Optimization</a></b>

<a id='sect1_1'></a>
### <font color='darkgreen'>Grid search</font>
<b><font size='3ptx'>The grid search is the most common hyperparameter tuning approach given its simple and straightforward procedure</font>. It is an `uninformed search method`, which means that it does not learn from its previous iterations.</b>

<b>Using this method entails testing every unique combination of hyperparameters in the search space to determine the combination that yields the best performance.</b> It’s easy to see the benefits of such a brute-force method; what better way to find the best solution than to try all of them out?

Unfortunately, this approach does not scale well; <b><font color='darkred'>an increase in the size of the hyperparameter search space will result in an exponential rise in run time and computation</font></b>.

<a id='sect1_2'></a>
### <font color='darkgreen'>Random search</font>
<b><font size='3ptx'>The random search is also an uninformed search method that treats iterations independently.</font></b> <b>However, instead of searching for all hyperparameter sets in the search space, it evaluates a specific number of hyperparameter sets at random. This number is determined by the user.</b>

Since it performs fewer trials in hyperparameter tuning, <b>the method requires less computation and run time than the grid search.</b>

Unfortunately, <b><font color='darkred'>since the random search tests hyperparameter sets at random, it runs the risk of missing the ideal set of hyperparameters and forgoing peak model performance</font></b>.

<a id='sect1_3'></a>
### <font color='darkgreen'>Bayesian Optimization</font>
<b><font size='3ptx'>Unlike the grid search and random search, which treat hyperparameter sets independently, the Bayesian optimization is an informed search method, meaning that it learns from previous iterations</font>. The number of trials in this approach is determined by the user.</b>

As the name suggests, the process is based on [**Bayes’ theorem**](https://en.wikipedia.org/wiki/Bayes%27_theorem):
> $P(A|B) = \frac{P(B|A) P(A)}{P(B)}$

<br/>

For this use case, the theorem can be modified to the following:
> $P(Score|Hyperparameters) = \frac{P(Hyperparameters|Score) P(Score)}{P(Hyperparameters)}$

<br/>

Simply put, <b>this method creates a probabilistic model, in which it maps hyperparameters to their corresponding score probability</b>.

Instead of painstakingly trying every hyperparameter set or testing hyperparameter sets at random, <b><font color='darkgreen'>the Bayesian optimization method can converge to the optimal hyperparameters. Thus, the best hyperparameters can be obtained without exploring the entire sample space</font></b>.

With the Bayesian optimization method, users do not have to endure long run times that come from evaluating every hyperparameter set. They also do not have to incorporate randomness and risk missing the optimal solution.

That being said, Bayesian optimization does have its own drawback. Since this is an informed learning method, additional time is required to determine the next hyperparameters to evaluate based on the results of the previous iterations. <b><font color='darkred'>At the expense of minimizing the number of trials, Bayesian optimization requires more time for each iteration.</font></b>

<a id='sect2'></a>
## <font color='darkblue'>Case Study</font>  ([back](#sect0))
* <b><a href='#sect2_1'>Grid Search</a></b>
* <b><a href='#sect2_2'>Random Search</a></b>
* <b><a href='#sect2_3'>Bayesian Optimization</a></b>
* <b><a href='#sect2_4'>Comparison</a></b>
<br/>

<b><font size='3ptx'>We have explored the ins and outs of the three hyperparameter tuning approaches. To consolidate our understanding of these methods, it is best to use an example.</font></b>

Let’s fine-tune a classification model with all three approaches and determine which one yields the best results.

For the exercise, we will use the [load digits dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html) from the Sklearn module.

In [11]:
import optuna 
import pandas as pd
import time

from optuna.samplers import TPESampler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
# load data
digits = datasets.load_digits()

# flatten the images
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Split data into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.25, shuffle=False)

The goal is to fine-tune a random forest model with the grid search, random search, and Bayesian optimization. Each method will be evaluated based on:
* The total number of trials executed
* The number of trials needed to yield the optimal hyperparameters
* The score of the model (<font color='brown'>F-1 score in this case</font>)
* The run time
<br/>

<b>The random forest classifier object and the search space</b> are shown below:

In [3]:
# random forest classifier object
rfc = RandomForestClassifier(random_state=42)

# define sample space
param_grid = {
    'n_estimators': [100,150,200],
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [5, 6, 7]}

Altogether, there are 810 unique hyperparameter combinations.

<a id='sect2_1'></a>
### <font color='darkgreen'>Grid Search</font> ([back](#sect2))
<b>First, let’s obtain the optimal hyperparameters using the grid search method and time the process. Of course, this means that <font size='3ptx'>we will test all 810 hyperparameter sets and pick out the one that yields the best results</font></b>.

In [6]:
%%time
# create grid search object
gs = GridSearchCV(estimator=rfc,
                  param_grid=param_grid,
                  scoring='f1_micro',
                  cv=5,
                  n_jobs=-1,
                  verbose=0)

# perform hyperparameter tuning (while timing the process)
time_start = time.time()
gs.fit(X_train, y_train)
time_grid = time.time() - time_start

# store result in a data frame 
values_grid = [810, gs.best_index_+1, gs.best_score_, time_grid]
columns = ['Number of iterations', 'Iteration Number of Optimal Hyperparamters', 'Score', 'Time Elapsed (s)']
results_grid = pd.DataFrame([values_grid], columns = columns)

CPU times: user 8.08 s, sys: 2.12 s, total: 10.2 s
Wall time: 8min 43s


<a id='sect2_2'></a>
### <font color='darkgreen'>Random Search</font> ([back](#sect2))
Next, we will use the random search to identify the optimal hyperparameters and time the process. The search is limited to 100 trials.

In [10]:
%%time
# create a random search object
rs = RandomizedSearchCV(estimator=rfc,
                  param_distributions=param_grid,
                  scoring='f1_micro',
                  cv=5,
                  n_jobs=-1,
                  verbose=0,
                  n_iter=100)

# perform hyperparamter tuning (while timing the process)
time_start = time.time()
rs.fit(X_train, y_train)
time_random = time.time() - time_start

# store result in a data frame 
values_grid = [[100, rs.best_index_+1, rs.best_score_, time_random]]
results_random = pd.DataFrame(values_grid, columns = columns)

CPU times: user 1.27 s, sys: 240 ms, total: 1.51 s
Wall time: 1min 1s


<a id='sect2_3'></a>
### <font color='darkgreen'>Bayesian Optimization</font> ([back](#sect2))
<b><font size='3ptx'>Finally, we perform hyperparameter tuning with the Bayesian optimization and time the process. In Python, this can be accomplished with the [Optuna module](https://optuna.org/)</font></b>.

Its syntax differs from that of Sklearn, but it performs the same operation. For the sake of consistency, we will use 100 trials in this procedure as well.

In [14]:
def objective(trial):
    """return the f1-score"""

    # search space
    n_estimators =  trial.suggest_int('n_estimators', low=100, high=200, step=50)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    min_samples_split = trial.suggest_int('min_samples_split', low=2, high=4, step=1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', low=1, high=5, step=1)
    max_depth = trial.suggest_int('max_depth', low=5, high=7, step=1)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt','log2'])

    # random forest classifier object
    rfc = RandomForestClassifier(
        n_estimators=n_estimators, 
        criterion=criterion,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_depth=max_depth,
        max_features=max_features,
        random_state=42)
    
    score =  cross_val_score(estimator=rfc, 
                             X=X_train, 
                             y=y_train, 
                             scoring='f1_micro',
                             cv=5,
                             n_jobs=-1).mean()
    
    return score

In [15]:
%%time
# create a study (aim to maximize score)
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(
    sampler=TPESampler(), direction='maximize')

# perform hyperparamter tuning (while timing the process)
time_start = time.time()
study.optimize(objective, n_trials=100)
time_bayesian = time.time() - time_start

# store result in a data frame 
values_bayesian = [100, study.best_trial.number, study.best_trial.value, time_bayesian]
results_bayesian = pd.DataFrame([values_bayesian], columns = columns)

CPU times: user 2.48 s, sys: 291 ms, total: 2.77 s
Wall time: 1min 35s


<a id='sect2_4'></a>
### <font color='darkgreen'>Comparison</font> ([back](#sect2))
<b><font size='3ptx'>Now that we have executed hyperparameter tuning with all three approaches, let’s see how the results of each method compare to each other.</font></b>

For convenience, we will store the results of all 3 hyperparameter tuning procedures in a single data frame.

In [16]:
df = results_grid.append(results_random).append(results_bayesian)
df.index = ['Grid Search', 'Random Search', 'Bayesian Optimization']
df

,Number of iterations,Iteration Number of Optimal Hyperparamters,Score,Time Elapsed (s)
Grid Search,810,680,0.935426,523.337782
Random Search,100,25,0.930965,61.371032
Bayesian Optimization,100,27,0.935426,95.283444


<b>The grid search registered the highest score</b> (<font color='brown'>joint with the Bayesian optimization method</font>). However, the method required carrying out 810 trials and only managed to obtain the optimal hyperparameters at the 680th iteration. Also, <b><font color='darkred'>its run time far exceeded that of the random search and the Bayesian optimization methods</font></b>.

<b>The random search method required only 100 trials and needed only 25 iterations to find the best hyperparameter set</b>. It also took the least amount of time to execute. However, <b><font color='darkred'>the random search method registered the lowest score out of the 3 methods</font></b>.

<b>The Bayesian optimization also performed 100 trials but was able to achieve the highest score after only 27 iterations, far less than the grid search’s 680 iterations</b>. Although it executed the same number of trials as the random search, it has a longer run time since it is an informed search method.

<a id='sect3'></a>
## <font color='darkblue'>Which method is the best?</font> ([back](#sect0))
<b><font size='3ptx'>Given that the grid search, random search, and Bayesian optimization all have their own trade-off between run time, the number of iterations, and performance, is it really possible to come to a consensus on which method is the best?</font></b>

Probably not.

After all, <b>the ideal hyperparameter tuning method depends on the use case</b>. Ask yourself:
* What are the constraints of your machine learning task?
* Does your project prioritize maximizing performance or minimizing run time and/or the number of iterations?

<br/>

Answering these questions will help decide the most suitable hyperparameter optimization approach. 

* **The grid search** is ideal if the computational demand and run-time are not limiting factors.
* **The random search** is suitable if you’re willing to sacrifice performance in exchange for fewer iterations and smaller run time (<font color='brown'>Theoretically, the random search could find the best hyperparameters, but this is left entirely up to chance</font>).
* **Bayesian optimization** is the best fit if you wish to obtain the optimal hyperparameters with fewer trials but are willing to have longer run times for each iteration.

### <font color='darkgreen'>My 2 cents</font>
<b><font size='3ptx'>If you hate diplomatic answers and just want my personal opinion, I would say that I usually favor the Bayesian optimization.</font></b>

<b>Given the run time needed for fine-tuning models with larger training data sets and search spaces, I usually shun the grid search.</b> The random search requires fewer iterations and is the fastest of all 3 methods, but its level of success depends on the hyperparameter sets that are selected at random. In some cases, it will select the optimal hyperparameters; in other cases, it will omit the optimal hyperparameters completely. Due to this inconsistency, I do not like relying on randomness for bigger machine learning tasks.

<b>I prefer the Bayesian optimization approach for its ability to consistently attain the optimal hyperparameters with fewer iterations.</b> Its individual iterations may take more time than those of the uninformed search methods, but that is rarely a deal-breaker for me.

### <font color='darkgreen'>Conclusion</font>
<b><font size='3ptx'>The main takeaway from this analysis is that each hyperparameter tuning method has its own unique trade-off between run time, number of iterations, and performance.</font></b>

Ultimately, the best approach for you will depend on your priorities and constraints. Having a strong understanding of the data as well as the objectives will ensure that you make the correct decision.

I wish you the best of luck in your data science endeavors!